In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

#Create directories

In [ ]:
TRAIN_DIR = 'images/training'
TEST_DIR = 'images/testing'

#Creating Datafram

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

#Creating Dataframe for Training

In [ ]:
training = pd.DataFrame()
training['image'], training['label'] = createdataframe(TRAIN_DIR)

In [ ]:
print(training)

#Creating Dataframe for testing

In [ ]:
testing = pd.DataFrame()
testing['image'], testing['label'] = createdataframe(TEST_DIR)

In [ ]:
print(testing)

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
training_features = extract_features(training['image'])

In [ ]:
testing_features = extract_features(testing['image'])

In [ ]:
x_training = training_features/255.0
x_testing = testing_features/255.0

#Creating label using sklearn model

In [ ]:
le = LabelEncoder()
le.fit(training['label'])

In [ ]:
y_training = le.transform(training['label'])
y_testing = le.transform(testing['label'])

In [ ]:
y_tarining = to_categorical(y_training,num_classes = 7)
y_testing = to_categorical(y_testing,num_classes = 7)

#Creating model

In [ ]:
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x = x_training,y = y_tarining, batch_size= 128, epochs = 100, validation_data = (x_testing, y_testing))

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

Testing

In [ ]:
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [ ]:
image = 'images/training/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)